# Graph of Thought (GoT) Prompting Example

This notebook demonstrates **Graph of Thought (GoT)**, one of the most advanced reasoning frameworks for LLMs.

**The Concept:**
Standard "Chain of Thought" is linear (Step 1 → Step 2 → Step 3). **Graph of Thought** models reasoning as a network (DAG - Directed Acyclic Graph).
* **Nodes**: Individual thoughts or intermediate steps.
* **Edges**: Relationships (e.g., "Step 1 *supports* Step 2", "Step 3 *contradicts* Step 1").
* **Transformations**: Thoughts can merge (aggregation), split (branching), or loop back (refinement).

**Key Mechanics:**
1.  **Prompting for Structure**: We force the LLM to output its reasoning as a **JSON** object containing `nodes` and `edges`.
2.  **Parsing**: We read this structured output to understand the relationship between thoughts.
3.  **Visualization**: We can potentially map or display this network to see how the model arrived at the answer.

In [ ]:
%pip install openai python-dotenv networkx matplotlib --quiet

### 1. Setup and Authorization

We import `openai` for the LLM, `json` for parsing the structured output, and `networkx` to manage the graph structure.

In [ ]:
from openai import OpenAI
import os
import json
import re
import networkx as nx
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

# Model configuration - can be overridden via environment variable
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

client = OpenAI(api_key=api_key)
print(f"OpenAI client ready! Using model: {MODEL}")

### 2. The GoT Function (The "Network Builder")

This function performs the heavy lifting:
1.  **Prompting**: It sends a strict instruction to the model to output JSON only.
2.  **Parsing**: It cleans the raw string (removing Markdown backticks) and converts it to a Python dictionary.
3.  **Graph Construction**: It uses the `networkx` library to actually build the graph object in memory, adding nodes and connecting them with edges.

In [ ]:
def clean_json_output(raw_content):
    """
    Clean model output to extract valid JSON.
    Handles common issues like markdown code blocks and extra text.
    """
    content = raw_content.strip()
    
    # Remove markdown code blocks (```json ... ``` or ``` ... ```)
    content = re.sub(r'^```(?:json)?\s*\n?', '', content)
    content = re.sub(r'\n?```\s*$', '', content)
    
    # Try to find JSON object boundaries if there's extra text
    json_match = re.search(r'\{[\s\S]*\}', content)
    if json_match:
        content = json_match.group(0)
    
    return content.strip()

def graph_of_thought(problem):
    print(f"Building reasoning graph for: '{problem}'...\n")
    
    try:
        # 1. Generate the Graph (JSON)
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{
                "role": "user", 
                "content": f"""
Solve this using Graph-of-Thought reasoning.
Problem: {problem}

You must output your reasoning structure as a valid JSON object.
Format:
{{
  "nodes": {{
    "1": "First thought or initial analysis",
    "2": "Another perspective or calculation",
    "3": "Merged insight from 1 and 2",
    "4": "Final Conclusion"
  }},
  "edges": [
    [1, 3, "supports"],
    [2, 3, "combines_with"],
    [3, 4, "leads_to"]
  ],
  "final_answer": "The concise answer here"
}}

Do not add any markdown formatting or explanation outside the JSON.
"""
            }],
            temperature=0.7 # Some creativity allowed for branching thoughts
        )
        
        raw_content = response.choices[0].message.content.strip()

        # 2. Parse JSON with robust cleaning
        clean_json = clean_json_output(raw_content)
        data = json.loads(clean_json)
        
        # 3. Build Graph Object
        G = nx.DiGraph()
        
        # Add Nodes
        print("Thoughts (Nodes):")
        for node_id, text in data["nodes"].items():
            G.add_node(node_id, label=text)
            print(f"  [{node_id}] {text}")
            
        # Add Edges
        print("\nRelationships (Edges):")
        for source, target, relation in data["edges"]:
            G.add_edge(str(source), str(target), label=relation)
            print(f"  [{source}] --({relation})--> [{target}]")
            
        print("\n" + "="*50)
        print(f"FINAL ANSWER: {data['final_answer']}")
        print("="*50)
        
        return G, data

    except json.JSONDecodeError as e:
        print(f"Error: The model failed to output valid JSON. {e}")
        print("Raw output:\n", raw_content if 'raw_content' in locals() else "No response")
        return None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None

### 3. Running the Graph of Thought

Now we test it. Try a complex problem that benefits from non-linear thinking, such as:
* "Write a story summary where the ending contradicts the beginning but makes sense."
* "Analyze the economic impact of AI, considering both job loss and productivity gains, and merge them."
* "Solve a riddle that requires combining two disparate clues."

In [ ]:
user_problem = input("Enter a complex problem: ")

if user_problem:
    graph, details = graph_of_thought(user_problem)
    
    if graph:
        print(f"\nGraph successfully built with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")